In [1]:
import os
import pandas as pd

In [2]:
os.listdir('../mimic-iv-1.0/mimic-iv-1.0/hosp')

['diagnoses_icd.csv.gz',
 'drgcodes.csv.gz',
 'd_hcpcs.csv.gz',
 'd_icd_diagnoses.csv.gz',
 'd_icd_procedures.csv.gz',
 'd_labitems.csv.gz',
 'emar.csv.gz',
 'emar_detail.csv.gz',
 'hcpcsevents.csv.gz',
 'labevents.csv.gz',
 'microbiologyevents.csv.gz',
 'pharmacy.csv.gz',
 'poe.csv.gz',
 'poe_detail.csv.gz',
 'prescriptions.csv.gz',
 'procedures_icd.csv.gz',
 'services.csv.gz']

In [3]:
os.listdir('../mimic-iv-1.0/mimic-iv-1.0/core')

['admissions.csv.gz', 'patients.csv.gz', 'transfers.csv.gz']

In [4]:
diagnosis_filepath = "../mimic-iv-1.0/mimic-iv-1.0/hosp/diagnoses_icd.csv.gz"
admission_filepath = "../mimic-iv-1.0/mimic-iv-1.0/core/admissions.csv.gz"
patients_filepath = "../mimic-iv-1.0/mimic-iv-1.0/core/patients.csv.gz"

In [5]:
df_ad = pd.read_csv(admission_filepath)
print(df_ad.shape)
df_ad['admittime'] = pd.to_datetime(df_ad['admittime'])
print(f"{df_ad['admittime'].min()} to {df_ad['admittime'].max()}")
df_ad.head()

(523740, 15)
2105-10-04 17:26:00 to 2212-04-06 12:13:00


,subject_id,hadm_id,admittime,dischtime,deathtime,admission_type,admission_location,discharge_location,insurance,language,marital_status,ethnicity,edregtime,edouttime,hospital_expire_flag
0,14679932,21038362,2139-09-26 14:16:00,2139-09-28 11:30:00,NaN,ELECTIVE,NaN,HOME,Other,ENGLISH,SINGLE,UNKNOWN,NaN,NaN,0
1,15585972,24941086,2123-10-07 23:56:00,2123-10-12 11:22:00,NaN,ELECTIVE,NaN,HOME,Other,ENGLISH,NaN,WHITE,NaN,NaN,0
2,11989120,21965160,2147-01-14 09:00:00,2147-01-17 14:25:00,NaN,ELECTIVE,NaN,HOME,Other,ENGLISH,NaN,UNKNOWN,NaN,NaN,0
3,17817079,24709883,2165-12-27 17:33:00,2165-12-31 21:18:00,NaN,ELECTIVE,NaN,HOME,Other,ENGLISH,NaN,OTHER,NaN,NaN,0
4,15078341,23272159,2122-08-28 08:48:00,2122-08-30 12:32:00,NaN,ELECTIVE,NaN,HOME,Other,ENGLISH,NaN,BLACK/AFRICAN AMERICAN,NaN,NaN,0


In [6]:
df_patients = pd.read_csv(patients_filepath)
print(df_patients.shape)
df_patients.head()

(382278, 6)


,subject_id,gender,anchor_age,anchor_year,anchor_year_group,dod
0,10000048,F,23,2126,2008 - 2010,NaN
1,10002723,F,0,2128,2017 - 2019,NaN
2,10003939,M,0,2184,2008 - 2010,NaN
3,10004222,M,0,2161,2014 - 2016,NaN
4,10005325,F,0,2154,2011 - 2013,NaN


In [7]:
df_patients['subject_id'].value_counts()

16919396    1
18873002    1
17773235    1
18811572    1
19970563    1
           ..
13143483    1
17345983    1
10180032    1
11341256    1
17072661    1
Name: subject_id, Length: 382278, dtype: int64

In [8]:
df_diag = pd.read_csv(diagnosis_filepath)
print(df_diag.shape)
df_diag.head()

(5280351, 5)


,subject_id,hadm_id,seq_num,icd_code,icd_version
0,15734973,20475282,3,2825,9
1,15734973,20475282,2,V0251,9
2,15734973,20475282,5,V270,9
3,15734973,20475282,1,64891,9
4,15734973,20475282,4,66481,9


In [9]:
df_diag['icd_code'].value_counts()

4019     104080
2724      68215
I10       54696
E785      51097
53081     49593
          ...  
1349          1
66983         1
96904         1
A082          1
C8445         1
Name: icd_code, Length: 27170, dtype: int64

In [10]:
df_diag_02 = df_diag.groupby(['subject_id', 'hadm_id']).agg({'icd_code': lambda x: '_'.join(x)}).reset_index()
df_diag_02['pri_icd'] = df_diag_02['icd_code'].apply(lambda x: x.split('_')[0])
df_diag_02['sec_icd'] = df_diag_02['icd_code'].apply(lambda x: '_'.join(x.split('_')[1:]))
df_diag_02['num_icd_code'] = df_diag_02['icd_code'].apply(lambda x: len(x.split('_')))
df_diag_02.head()

,subject_id,hadm_id,icd_code,pri_icd,sec_icd,num_icd_code
0,10000019,25058216,V053_V3000_V290_V502,V053,V3000_V290_V502,4
1,10000032,22595853,496_07070_29680_V1582_5715_78959_5723_30981,496,07070_29680_V1582_5715_78959_5723_30981,8
2,10000032,22841357,5715_2875_07071_V08_2761_78959_496_3051,5715,2875_07071_V08_2761_78959_496_3051,8
3,10000032,25742920,V08_2761_2767_3051_V462_78791_496_5715_78959_0...,V08,2761_2767_3051_V462_78791_496_5715_78959_07054,10
4,10000032,29079034,496_V08_V462_45829_3051_07044_7994_5715_V4986_...,496,V08_V462_45829_3051_07044_7994_5715_V4986_2767...,13


In [11]:
df_valid_ad = pd.merge(df_ad, df_diag_02, how='inner', on=['subject_id', 'hadm_id'])
print(df_ad.shape)
print(df_valid_ad.shape)
df_valid_ad.head()

(523740, 15)
(521110, 19)


,subject_id,hadm_id,admittime,dischtime,deathtime,admission_type,admission_location,discharge_location,insurance,language,marital_status,ethnicity,edregtime,edouttime,hospital_expire_flag,icd_code,pri_icd,sec_icd,num_icd_code
0,14679932,21038362,2139-09-26 14:16:00,2139-09-28 11:30:00,NaN,ELECTIVE,NaN,HOME,Other,ENGLISH,SINGLE,UNKNOWN,NaN,NaN,0,V053_V3000_7500,V053,V3000_7500,3
1,15585972,24941086,2123-10-07 23:56:00,2123-10-12 11:22:00,NaN,ELECTIVE,NaN,HOME,Other,ENGLISH,NaN,WHITE,NaN,NaN,0,V3101_V053,V3101,V053,2
2,11989120,21965160,2147-01-14 09:00:00,2147-01-17 14:25:00,NaN,ELECTIVE,NaN,HOME,Other,ENGLISH,NaN,UNKNOWN,NaN,NaN,0,Z3801_P599_Z23,Z3801,P599_Z23,3
3,17817079,24709883,2165-12-27 17:33:00,2165-12-31 21:18:00,NaN,ELECTIVE,NaN,HOME,Other,ENGLISH,NaN,OTHER,NaN,NaN,0,P221_P0739_Z23_Z3831_Z051,P221,P0739_Z23_Z3831_Z051,5
4,15078341,23272159,2122-08-28 08:48:00,2122-08-30 12:32:00,NaN,ELECTIVE,NaN,HOME,Other,ENGLISH,NaN,BLACK/AFRICAN AMERICAN,NaN,NaN,0,V502_V290_V3000,V502,V290_V3000,3


In [12]:
df_ad_visits = df_valid_ad.groupby(['subject_id']).agg({'hadm_id': 'count'}).reset_index()
print(f"Before: {df_ad_visits.shape}")
df_ad_visits = df_ad_visits.loc[(df_ad_visits['hadm_id']>2)]
print(f"After: {df_ad_visits.shape}")
df_ad_visits.head()

Before: (255106, 2)
After: (46389, 2)


,subject_id,hadm_id
1,10000032,4
21,10000826,3
25,10000935,5
26,10000980,7
28,10001180,3


In [13]:
print(f"Before: {df_valid_ad.shape}")
df_valid_ad = df_valid_ad.loc[(df_valid_ad['subject_id'].isin(df_ad_visits['subject_id'].tolist()))]
print(f"After: {df_valid_ad.shape}")
df_valid_ad.head()

Before: (521110, 19)
After: (273489, 19)


,subject_id,hadm_id,admittime,dischtime,deathtime,admission_type,admission_location,discharge_location,insurance,language,marital_status,ethnicity,edregtime,edouttime,hospital_expire_flag,icd_code,pri_icd,sec_icd,num_icd_code
55,19120008,24459786,2185-09-18 11:15:00,2185-09-20 15:30:00,NaN,SURGICAL SAME DAY ADMISSION,PHYSICIAN REFERRAL,HOME,Medicare,?,MARRIED,ASIAN,NaN,NaN,0,4019_2230_V1209,4019,2230_V1209,3
62,16261811,26233676,2145-12-08 18:41:00,2145-12-09 19:40:00,NaN,EU OBSERVATION,EMERGENCY ROOM,NaN,Medicare,ENGLISH,SINGLE,WHITE,2145-12-08 14:44:00,2145-12-08 19:48:00,0,V4582_41401_4142_3003_412_78650_4019,V4582,41401_4142_3003_412_78650_4019,7
63,12988422,25192155,2132-05-24 07:10:00,2132-05-24 13:50:00,NaN,EU OBSERVATION,EMERGENCY ROOM,NaN,Medicare,ENGLISH,SINGLE,WHITE,2132-05-23 22:09:00,2132-05-24 13:50:00,0,29640_7197_3079_30500,29640,7197_3079_30500,4
65,15772648,20829310,2145-09-01 19:50:00,2145-09-04 14:35:00,NaN,EW EMER.,EMERGENCY ROOM,HOME,Medicare,ENGLISH,SINGLE,WHITE,NaN,NaN,0,4019_V694_07070_71180_29680_7801_V6284_3051_15...,4019,V694_07070_71180_29680_7801_V6284_3051_1550_30...,13
132,11309329,25435133,2128-11-27 18:41:00,2128-11-27 21:50:00,NaN,EU OBSERVATION,EMERGENCY ROOM,NaN,Other,ENGLISH,MARRIED,WHITE,2128-11-27 13:38:00,2128-11-27 21:50:00,0,I509_N390_I959_E860_E119_I10_Z794_I5020_I2510_...,I509,N390_I959_E860_E119_I10_Z794_I5020_I2510_C569,10


In [14]:
df_valid_ad = df_valid_ad[['subject_id', 'hadm_id', 'admittime', 'dischtime', 'ethnicity', 'pri_icd', 'sec_icd', 'icd_code']]
df_valid_ad = pd.merge(df_valid_ad, df_patients[['subject_id', 'gender', 'anchor_age']])
df_valid_ad.sort_values(by=['admittime'], ascending=True, inplace=True)

In [15]:
df_valid_ad.head()

,subject_id,hadm_id,admittime,dischtime,ethnicity,pri_icd,sec_icd,icd_code,gender,anchor_age
2346,16233333,26733622,2109-08-31 04:20:00,2109-08-31 07:51:00,WHITE,27651,30500_042,27651_30500_042,M,47
271257,12024697,20302177,2109-12-14 22:50:00,2110-01-15 14:53:00,WHITE,2859,78551_99859_496_5849_72889_25000_5990_42731_42...,2859_78551_99859_496_5849_72889_25000_5990_427...,M,83
200109,17195991,23542772,2110-01-11 22:47:00,2110-01-18 10:25:00,UNABLE TO OBTAIN,5990,V1042_35800_29620_70722_70703_2768_V4986_4019_...,5990_V1042_35800_29620_70722_70703_2768_V4986_...,F,61
22557,15918556,27863083,2110-01-13 20:03:00,2110-01-14 15:04:00,WHITE,V1588,49390_41400_3659_4280_340_72989_42822,V1588_49390_41400_3659_4280_340_72989_42822,M,77
162917,10390531,29125241,2110-01-16 02:41:00,2110-01-20 11:18:00,WHITE,E8801,70722_70703_27652_45829_E8781_V4365_82101_2851...,E8801_70722_70703_27652_45829_E8781_V4365_8210...,F,74


In [16]:
df_train = df_valid_ad.groupby(['subject_id'], as_index=False).apply(lambda x: x.iloc[:-1]).reset_index(drop=True)
df_train.head()

,subject_id,hadm_id,admittime,dischtime,ethnicity,pri_icd,sec_icd,icd_code,gender,anchor_age
0,10000032,22595853,2180-05-06 22:23:00,2180-05-07 17:15:00,WHITE,496,07070_29680_V1582_5715_78959_5723_30981,496_07070_29680_V1582_5715_78959_5723_30981,F,52
1,10000032,22841357,2180-06-26 18:27:00,2180-06-27 18:49:00,WHITE,5715,2875_07071_V08_2761_78959_496_3051,5715_2875_07071_V08_2761_78959_496_3051,F,52
2,10000032,29079034,2180-07-23 12:35:00,2180-07-25 17:55:00,WHITE,496,V08_V462_45829_3051_07044_7994_5715_V4986_2767...,496_V08_V462_45829_3051_07044_7994_5715_V4986_...,F,52
3,10000826,20032235,2146-12-05 19:07:00,2146-12-12 16:30:00,WHITE,5990,2768_30391_2639_5712_56409_5723_2761_486_78959...,5990_2768_30391_2639_5712_56409_5723_2761_486_...,F,32
4,10000826,21086876,2146-12-18 17:39:00,2146-12-24 19:55:00,WHITE,99591,2761_3051_5990_5119_30391_5710_78959_5711,99591_2761_3051_5990_5119_30391_5710_78959_5711,F,32


In [17]:
df_test = df_valid_ad.groupby(['subject_id'], as_index=False).apply(lambda x: x.iloc[-1]).reset_index(drop=True)
df_test.head()

,subject_id,hadm_id,admittime,dischtime,ethnicity,pri_icd,sec_icd,icd_code,gender,anchor_age
0,10000032,25742920,2180-08-05 23:44:00,2180-08-07 17:50:00,WHITE,V08,2761_2767_3051_V462_78791_496_5715_78959_07054,V08_2761_2767_3051_V462_78791_496_5715_78959_0...,F,52
1,10000826,28289260,2146-12-31 00:43:00,2147-01-02 17:45:00,WHITE,78959,5711_5723_2761_33829_2875_7242_5712,78959_5711_5723_2761_33829_2875_7242_5712,F,32
2,10000935,25849114,2187-10-10 19:09:00,2187-10-26 17:00:00,BLACK/AFRICAN AMERICAN,V4572,311_2869_5780_1970_1977_1539_V1582_42789_72989...,V4572_311_2869_5780_1970_1977_1539_V1582_42789...,F,52
3,10000980,20897796,2193-08-15 01:01:00,2193-08-17 15:07:00,BLACK/AFRICAN AMERICAN,I130,Z794_E1151_Z6831_N2581_M1A9XX0_I2510_E785_D648...,I130_Z794_E1151_Z6831_N2581_M1A9XX0_I2510_E785...,F,73
4,10001180,27864856,2197-06-06 08:12:00,2197-06-10 10:15:00,WHITE,N858,E109_F419_Z3A37_Z794_J329_O34211_O99284_O9952_...,N858_E109_F419_Z3A37_Z794_J329_O34211_O99284_O...,F,33


In [18]:
df_valid_ad.loc[(df_valid_ad['subject_id']==10000032)]

,subject_id,hadm_id,admittime,dischtime,ethnicity,pri_icd,sec_icd,icd_code,gender,anchor_age
236588,10000032,22595853,2180-05-06 22:23:00,2180-05-07 17:15:00,WHITE,496,07070_29680_V1582_5715_78959_5723_30981,496_07070_29680_V1582_5715_78959_5723_30981,F,52
236589,10000032,22841357,2180-06-26 18:27:00,2180-06-27 18:49:00,WHITE,5715,2875_07071_V08_2761_78959_496_3051,5715_2875_07071_V08_2761_78959_496_3051,F,52
236587,10000032,29079034,2180-07-23 12:35:00,2180-07-25 17:55:00,WHITE,496,V08_V462_45829_3051_07044_7994_5715_V4986_2767...,496_V08_V462_45829_3051_07044_7994_5715_V4986_...,F,52
236590,10000032,25742920,2180-08-05 23:44:00,2180-08-07 17:50:00,WHITE,V08,2761_2767_3051_V462_78791_496_5715_78959_07054,V08_2761_2767_3051_V462_78791_496_5715_78959_0...,F,52


In [19]:
df_train.loc[(df_train['subject_id']==10000032)]

,subject_id,hadm_id,admittime,dischtime,ethnicity,pri_icd,sec_icd,icd_code,gender,anchor_age
0,10000032,22595853,2180-05-06 22:23:00,2180-05-07 17:15:00,WHITE,496,07070_29680_V1582_5715_78959_5723_30981,496_07070_29680_V1582_5715_78959_5723_30981,F,52
1,10000032,22841357,2180-06-26 18:27:00,2180-06-27 18:49:00,WHITE,5715,2875_07071_V08_2761_78959_496_3051,5715_2875_07071_V08_2761_78959_496_3051,F,52
2,10000032,29079034,2180-07-23 12:35:00,2180-07-25 17:55:00,WHITE,496,V08_V462_45829_3051_07044_7994_5715_V4986_2767...,496_V08_V462_45829_3051_07044_7994_5715_V4986_...,F,52


In [20]:
df_train.loc[(df_train['subject_id']==10000032)]

,subject_id,hadm_id,admittime,dischtime,ethnicity,pri_icd,sec_icd,icd_code,gender,anchor_age
0,10000032,22595853,2180-05-06 22:23:00,2180-05-07 17:15:00,WHITE,496,07070_29680_V1582_5715_78959_5723_30981,496_07070_29680_V1582_5715_78959_5723_30981,F,52
1,10000032,22841357,2180-06-26 18:27:00,2180-06-27 18:49:00,WHITE,5715,2875_07071_V08_2761_78959_496_3051,5715_2875_07071_V08_2761_78959_496_3051,F,52
2,10000032,29079034,2180-07-23 12:35:00,2180-07-25 17:55:00,WHITE,496,V08_V462_45829_3051_07044_7994_5715_V4986_2767...,496_V08_V462_45829_3051_07044_7994_5715_V4986_...,F,52
